# TTR - Lab 4 - Simpy - Célestin Piccin

In [1]:
#pip install simpy

## Tutorial

In [2]:
import simpy

def car(env):
    while True:
        print('Start parking at %d' % env.now)
        parking_duration = 5
        yield env.timeout(parking_duration)

        print('Start driving at %d' % env.now)
        trip_duration = 2
        yield env.timeout(trip_duration)
        

env = simpy.Environment()
env.process(car(env))
env.run(until=15)

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14


## Case 1a: Base scenario M/M/1

In [3]:
# SimPy simulation of an M/M/1 queueing system.
# The system has a single server and an infinite queue.
# The inter-arrival time is exponentially distributed with a mean of 1.0 time units.
# The service time is exponentially distributed with a mean of 0.5 time units.
# The simulation should run for 1000 time units.
# The output should be an array of the service times for each customer.

from statistics import mean
import random
import numpy as np

# Parameters
arrival_rate = 90.0
service_rate = 100.0
num_requests = 1_000_000

# Results
response_times = []

# ---------------------------------------------------------------------------
# SimPy model
env = simpy.Environment()
server = simpy.Resource(env, capacity=1)

def request_generator(env):
    for i in range(num_requests):
        yield env.timeout(random.expovariate(arrival_rate))
        env.process(process_request(env))

def process_request(env):
    arrival_time = env.now
    job = server.request()
    # Wait for the server to become available (wait in the queue)
    yield job
    # Process the request
    yield env.timeout(random.expovariate(service_rate))
    departure_time = env.now
    response_times.append(departure_time - arrival_time)
    server.release(job)

env.process(request_generator(env))
env.run()

# ---------------------------------------------------------------------------
# Compute the results
mean_response_time = np.mean(response_times)
print(f'Mean response time: {mean_response_time:.4f} s')

Mean response time: 0.1023 s


> For mean time of 100 ms a mu of 100 is sufficient.

## Case 1b: M/M/1 model when doubling the arrival rate

In [4]:
# Parameters
arrival_rate = 180.0
service_rate = 190.0
num_requests = 1_000_000

# Results
response_times = []

env = simpy.Environment()
server = simpy.Resource(env, capacity=1)
env.process(request_generator(env))
env.run()

# ---------------------------------------------------------------------------
# Compute the results
mean_response_time = np.mean(response_times)
print(f'Mean response time: {mean_response_time:.4f} s')

Mean response time: 0.1030 s


> To keep a mean time of 100ms we do not have to double mu in fact putting mu = 190 is sufficient. We also see that the simulated results obtained are almost identical to the analytical ones.

## Case 2a: Batch arrivals

> First we need to compute the new lambda. 

> Since each web page request generates a random number of file downloads (uniformly distributed between 1 and 9), the average number of file downloads per web page is (1 + 9)/2 = 5. Thus, to achieve a file download rate of 90 per second we need to set lambda = 18 because 18 * 5 = 90

In [5]:
# Parameters
arrival_rate_webpages = 18.0
service_rate = 127.0
num_requests = 1_000_000

# Results
response_times = []

# ---------------------------------------------------------------------------
# SimPy model
env = simpy.Environment()
server = simpy.Resource(env, capacity=1)

def request_generator(env):
    for i in range(num_requests):
        # Wait for the next web page request
        yield env.timeout(random.expovariate(arrival_rate_webpages))
        # Generate a batch of file download requests (1-9 requests per web page)
        batch_size = random.randint(1, 9)
        for _ in range(batch_size):
            env.process(process_request(env))

def process_request(env):
    arrival_time = env.now
    job = server.request()
    # Wait for the server to become available (wait in the queue)
    yield job
    # Process the request
    yield env.timeout(random.expovariate(service_rate))
    departure_time = env.now
    response_times.append(departure_time - arrival_time)
    server.release(job)

env.process(request_generator(env))
env.run()

# ---------------------------------------------------------------------------
# Compute the results
mean_response_time = np.mean(response_times)
print(f'Mean response time: {mean_response_time:.4f} s')

Mean response time: 0.0992 s


## Case 2b: Batch arrivals and double arrival rate

> If we double the arrival of web requests then lambda = 36 because 18 * 2 = 36 and as we have seen in 2a 36 * 5 = 180 which corresponds to the doubling of the arrival rate like we saw in 1b.

In [6]:
# Parameters
arrival_rate_webpages = 36.0
service_rate = 217.0
num_requests = 1_000_000

# Results
response_times = []

# ---------------------------------------------------------------------------
# SimPy model
env = simpy.Environment()
server = simpy.Resource(env, capacity=1)
env.process(request_generator(env))
env.run()

# ---------------------------------------------------------------------------
# Compute the results
mean_response_time = np.mean(response_times)
print(f'Mean response time: {mean_response_time:.4f} s')

Mean response time: 0.0995 s


## Case 3a: Batch arrivals and 99th percentile

> As in case 2a we will use a lambda of 18. 

In [7]:
# Parameters
arrival_rate_webpages = 18.0
service_rate = 143.0
num_requests = 1_000_000

# Results
response_times = []

# ---------------------------------------------------------------------------
# SimPy model
env = simpy.Environment()
server = simpy.Resource(env, capacity=1)
env.process(request_generator(env))
env.run()

# ---------------------------------------------------------------------------
# Compute the results
response_time_99 = np.percentile(response_times, 99)
print(f'Response time (99th percentile): {response_time_99:.4f} s')

Response time (99th percentile): 0.2972 s


## Case 3b: Batch arrivals, 99th percentile and double arrival rate

> As in case 2b we will use a lambda of 36.

In [8]:
# Parameters
arrival_rate_webpages = 36.0
service_rate = 234.0
num_requests = 1_000_000

# Results
response_times = []

# ---------------------------------------------------------------------------
# SimPy model
env = simpy.Environment()
server = simpy.Resource(env, capacity=1)
env.process(request_generator(env))
env.run()

# ---------------------------------------------------------------------------
# Compute the results
response_time_99 = np.percentile(response_times, 99)
print(f'Response time (99th percentile): {response_time_99:.4f} s')

Response time (99th percentile): 0.3058 s


## Visualization of the response time distribution

In [9]:
# Response time plots
import json
import matplotlib.pyplot as plt
import numpy as np

MAX_POINTS = 1_000_000


# ----------------------------------------------------------------------------
# Histogram plot
def histogram_plot(data, max_points=MAX_POINTS, num_bins=100,
                   title="Histogram", xlabel="Response time (s)",
                   filename='histogram.png'):
    d = data[:max_points]
    plt.hist(d, bins=num_bins, color='steelblue', edgecolor='black')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.savefig(filename, bbox_inches="tight")
    plt.close()

# ----------------------------------------------------------------------------
# Scatter plot
def scatter_plot(data, max_points=MAX_POINTS,
                 title="Scatter plot", ylabel="Response time (s)",
                 filename='scatter.png'):
    d = data[:max_points]
    fig = plt.scatter(x=range(len(d)), y=d, s=1, color='steelblue')
    plt.title(title)
    plt.ylabel(ylabel)
    plt.savefig(filename, bbox_inches="tight")
    plt.close()

# ----------------------------------------------------------------------------
# Percentile plot
def percentile_plot(data, max_points=MAX_POINTS, window=10_000,
                    title="Percentile plot", ylabel="Response time (s)",
                    filename='percentiles.png'):
    d = data[:max_points]
    windows = np.reshape(d, (-1, window))
    dmean = np.mean(windows, axis=1)
    dmedian = np.median(windows, axis=1)
    d95 = np.percentile(windows, 95, axis=1)
    d99 = np.percentile(windows, 99, axis=1)

    # Plot all series in a line plot
    plt.plot(dmean, label='Mean', color='steelblue', linestyle='-')
    plt.plot(dmedian, label='Median', color='steelblue', linestyle='--')
    plt.plot(d95, label='95th percentile', color='steelblue', linestyle='-.')
    plt.plot(d99, label='99th percentile', color='steelblue', linestyle=':')

    xticks = np.arange(0, max_points//window, 4)
    plt.xticks(ticks=xticks, labels=xticks * window, rotation=90)
    plt.title(title)
    plt.ylabel(ylabel)
    plt.legend(loc='upper right')
    plt.savefig(filename, bbox_inches="tight")
    plt.close()

# ----------------------------------------------------------------------------
# Heatmap plot
def heatmap_plot(data, max_points=MAX_POINTS, x_points=20, y_points=40,
                 title="Heatmap plot", ylabel="Response time (s)",
                 filename='heatmap.png'):
    d = data[:max_points]
    window = max_points//x_points
    num_bins = y_points
    d_min = np.min(d)
    d_max = np.max(d)
    windows = np.reshape(d, (-1, window))
    histograms = np.array([np.histogram(window, bins=num_bins, range=(d_min, d_max))[0] for window in windows])
    bin_edges = np.histogram_bin_edges(data, bins=num_bins, range=(data.min(), data.max()))
    plt.imshow(histograms.T, aspect='auto', cmap='Reds', origin='lower')
    plt.colorbar(label='Number of values')
    xticks = np.arange(0, x_points, 2)
    plt.xticks(ticks=xticks, labels=xticks * window, rotation=90)
    yticks = np.arange(0, num_bins, 4)
    plt.yticks(ticks=yticks, labels=np.round(yticks*(d_max-d_min)/num_bins, 2))
    plt.title(title)
    plt.ylabel(ylabel)
    plt.savefig(filename, bbox_inches="tight")
    plt.close()

In [10]:
response_times_np_array = np.array(response_times)

histogram_plot(response_times_np_array)

scatter_plot(response_times_np_array)

percentile_plot(response_times_np_array)

heatmap_plot(response_times_np_array)

> After executing the above code, the resulting plots are saved as a PNG and have been integrated with their interpretation in the Report.md file